## BaseModel in Pydantic

### Test_1

In [1]:
from pydantic import BaseModel

# Define a User model using BaseModel
class User(BaseModel):
    id: int
    name: str
    age: int
    email: str

# Create an instance of the User model
user_data = {
    "id": 1,
    "name": "John Doe",
    "age": 30,
    "email": "john.doe@example.com"
}

user = User(**user_data)

# Convert the User model to a dictionary using model_dump()
user_dict = user.model_dump()
print(user_dict)
# Output: {'id': 1, 'name': 'John Doe', 'age': 30, 'email': 'john.doe@example.com'}

# Convert the User model to JSON using model_dump_json()
user_json = user.model_dump_json()
print(user_json)
# Output: {"id":1,"name":"John Doe","age":30,"email":"john.doe@example.com"}

{'id': 1, 'name': 'John Doe', 'age': 30, 'email': 'john.doe@example.com'}
{"id":1,"name":"John Doe","age":30,"email":"john.doe@example.com"}


### Test_2 (advance)

In [2]:
from pydantic import BaseModel, field_validator, Field, ValidationError
from typing import Optional, List
from enum import Enum

# Define an Enum for user roles
class Role(str, Enum):
    ADMIN = "admin"
    EDITOR = "editor"
    VIEWER = "viewer"

# Define a nested Address model
class Address(BaseModel):
    street: str
    city: str
    state: str
    zip_code: str = Field(..., alias="zipCode")  # Use alias for JSON/dictionary keys

    # Custom validator to ensure the zip code is 5 digits
    @field_validator("zip_code")
    @classmethod
    def validate_zip_code(cls, value):
        if not value.isdigit() or len(value) != 5:
            raise ValueError("Zip code must be 5 digits")
        return value

# Define the main User model
class User(BaseModel):
    id: int
    name: str
    age: Optional[int] = None  # Optional field
    email: str
    role: Role = Role.VIEWER  # Default value for role
    addresses: List[Address] = []  # List of nested Address models

    # Custom validator to ensure age is positive
    @field_validator("age")
    @classmethod
    def validate_age(cls, value):
        if value is not None and value < 0:
            raise ValueError("Age must be a positive number")
        return value

# Example usage
user_data = {
    "id": 1,
    "name": "John Doe",
    "email": "john.doe@example.com",
    "role": "admin",  # Enum value
    "addresses": [
        {
            "street": "123 Main St",
            "city": "New York",
            "state": "NY",
            "zipCode": "10001"  # Using alias
        }
    ]
}

# Create a User instance
user = User(**user_data)

# Print the user model
print(user)
# Output: id=1 name='John Doe' age=None email='john.doe@example.com' role=<Role.ADMIN: 'admin'> addresses=[Address(street='123 Main St', city='New York', state='NY', zip_code='10001')]

# Convert the User model to a dictionary
user_dict = user.model_dump(by_alias=True)  # Use aliases for dictionary keys
print(user_dict)
# Output: {'id': 1, 'name': 'John Doe', 'age': None, 'email': 'john.doe@example.com', 'role': 'admin', 'addresses': [{'street': '123 Main St', 'city': 'New York', 'state': 'NY', 'zipCode': '10001'}]}

# Convert the User model to JSON
user_json = user.model_dump_json(by_alias=True)
print(user_json)
# Output: {"id":1,"name":"John Doe","age":null,"email":"john.doe@example.com","role":"admin","addresses":[{"street":"123 Main St","city":"New York","state":"NY","zipCode":"10001"}]}

# Validation error example
try:
    invalid_user_data = {
        "id": 2,
        "name": "Jane Doe",
        "email": "jane.doe@example.com",
        "role": "invalid_role",  # Invalid enum value
        "addresses": [
            {
                "street": "456 Elm St",
                "city": "Los Angeles",
                "state": "CA",
                "zipCode": "9000A"  # Invalid zip code
            }
        ]
    }
    invalid_user = User(**invalid_user_data)
except ValidationError as e:
    print(f"Validation error: {e}")
# Output: Validation error: 2 validation errors for User
# role: Input should be 'admin', 'editor' or 'viewer' [type=enum, input_value='invalid_role', input_type=str]
# addresses.0.zipCode: Value error, Zip code must be 5 digits [type=value_error, input_value='9000A', input_type=str]

id=1 name='John Doe' age=None email='john.doe@example.com' role=<Role.ADMIN: 'admin'> addresses=[Address(street='123 Main St', city='New York', state='NY', zip_code='10001')]
{'id': 1, 'name': 'John Doe', 'age': None, 'email': 'john.doe@example.com', 'role': <Role.ADMIN: 'admin'>, 'addresses': [{'street': '123 Main St', 'city': 'New York', 'state': 'NY', 'zipCode': '10001'}]}
{"id":1,"name":"John Doe","age":null,"email":"john.doe@example.com","role":"admin","addresses":[{"street":"123 Main St","city":"New York","state":"NY","zipCode":"10001"}]}
Validation error: 2 validation errors for User
role
  Input should be 'admin', 'editor' or 'viewer' [type=enum, input_value='invalid_role', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/enum
addresses.0.zipCode
  Value error, Zip code must be 5 digits [type=value_error, input_value='9000A', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error
